# The process of Data Cleaning and Data Transformation

First remove duplicates:

In [ ]:
#import pandas
import pandas as pd
import numpy as np
import csv

In [ ]:
home_df = pd.read_csv('Home0_563.csv', encoding='utf-8').drop_duplicates()

In [ ]:
hdf = home_df.iloc[:,:26]
len(hdf)

In [ ]:
hdf.url.duplicated().sum()

In [ ]:
hdf = hdf.drop_duplicates(subset='url', keep='last', ignore_index=True)

In [ ]:
display(hdf)

Extracting Price from Price field

In [ ]:
p = hdf['price'].str
hdf['price'] = np.where(p.startswith('$'),p.replace(',','',regex=True), p)
hdf['price'] = np.where(p.startswith('$'),p.replace(' ','',regex=True), p)
hdf['price'] = np.where(p.startswith('$'),p.replace('$','',regex=True), p)
print(hdf['price'])

In [ ]:
display(hdf.iloc[0,26])

In [ ]:
hdf.shape

Spliting the 3 frist digit from postal code and creating a new column

In [ ]:
count = 0
hdf['po3'] = hdf['po']
for x in range(510):
    s = hdf.iloc[count][5]
    hdf['po3'][count] = s[:3] 
    print(count, s[:3])
    count = count +1
print(hdf['po3'])

In [ ]:
display(hdf)

Extracting Total finished Area from related field

In [ ]:
p = hdf['totalFinishedArea'].str
hdf['totalFinishedArea'] = np.where(p.endswith('sqft'),p.replace(' sqft','',regex=True), p)

Extracting Price History for calculating annual growth rate (CAGR)

In [ ]:
py = hdf['PriceHistory'][0]
py = py[py.find('$')+1:]
py = py.replace(',','')
py_int = int(py)
print(py_int)

In [ ]:
print(hdf.iloc[0][19])

In [ ]:
y = hdf['PriceHistory'][0]
loc = y.find('Sold')
y = y[loc-5:loc-1]
y_int = int(y)
print(loc)
print(y_int)

In [ ]:
pn_int = int(hdf['price'][0])
print(pn_int)

In [ ]:
CAGR = (((pn_int/py_int)**(1/(2022-y_int)))-1)
print(CAGR)

In [ ]:
count = 0
py = ''
y = ''
pn = ''
CAGR = np.zeros(510)
while count<510:
    s = str(hdf['PriceHistory'][count])
    if (s != 'nan'):
        if s.find('$') != -1:
            py = s[s.find('$')+1:]            
            py = py.replace(' (CAD)','')
            py = py.replace(',','')
            py_int = int(py)
            loc = s.find('Sold')
            y = s[loc-5:loc-1]
            y_int = int(y)
            pn_int = int(hdf['price'][count])
            CAGR[count] = (((pn_int/py_int)**(1/(2022-y_int)))-1)
    else:
        py = ''
        py_int = 0
        pn = ''
        pn_int = 0
        y = '0'
        y_int = 0
    print([count, s, py_int , pn_int, y_int, CAGR[count]])
    count = count +1


In [ ]:
hdf['CAGR'] = CAGR
print(hdf['CAGR'])

In [ ]:
hdf.head()

Write the result to a new file

In [ ]:
hdf.to_csv('home0_510.csv', index=False)